# Industry 4.0 의 중심, AI - ML&DL

<div align='right'><font size=2 color='gray'>Machine Learning & Deep Learning with TensorFlow @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, 김진수</font></div>
<hr>

# Sect12. RNN (Recurrent Newural Network)

In [1]:
from images import bigpycraft_ai as bpc
from IPython.display import Image 
from tqdm import tqdm_notebook

In [2]:
import time
import os

def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time

### 12-4. rnn_long_sentence

<hr>
``` python 

# Lab 12 RNN
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
tf.set_random_seed(777)  # reproducibility

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
seq_length = 10  # Any arbitrary number

dataX = []
dataY = []
for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1: i + seq_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape

# Make a lstm cell with hidden_size (each unit output vector size)
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, dtype=tf.float32)

# (optional) softmax layer
X_for_softmax = tf.reshape(outputs, [-1, hidden_size])
softmax_w = tf.get_variable("softmax_w", [hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])
# All weights are 1 (equal weights)
weights = tf.ones([batch_size, seq_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(mean_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

'''
0 167 tttttttttt 3.23111
0 168 tttttttttt 3.23111
0 169 tttttttttt 3.23111
…
499 167  of the se 0.229616
499 168 tf the sea 0.229616
499 169   the sea. 0.229616

g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

'''



```

<hr>

In [3]:
# Lab 12 RNN
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

tf.set_random_seed(777)  # reproducibility


C:\Python\Anaconda3-50\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
sentence = ("""\
당신이 배를 만들고 싶다면, 사람들에게 목재를 가져오게 하고 일을 지시하고 일감을 나눠주는 일을 하지 말라. \
대신 그들에게 저 넓고 끝없는 바다에 대한 동경심을 키워줘라.\
""")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}
len(char_set)

49

In [5]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
seq_length = 10  # Any arbitrary number

data_dim, hidden_size, num_classes

(49, 49, 49)

In [6]:
dataX = []
dataY = []

for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1: i + seq_length + 1]
    print(i, x_str, '\t->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)
    

0 당신이 배를 만들고 	-> 신이 배를 만들고 
1 신이 배를 만들고  	-> 이 배를 만들고 싶
2 이 배를 만들고 싶 	->  배를 만들고 싶다
3  배를 만들고 싶다 	-> 배를 만들고 싶다면
4 배를 만들고 싶다면 	-> 를 만들고 싶다면,
5 를 만들고 싶다면, 	->  만들고 싶다면, 
6  만들고 싶다면,  	-> 만들고 싶다면, 사
7 만들고 싶다면, 사 	-> 들고 싶다면, 사람
8 들고 싶다면, 사람 	-> 고 싶다면, 사람들
9 고 싶다면, 사람들 	->  싶다면, 사람들에
10  싶다면, 사람들에 	-> 싶다면, 사람들에게
11 싶다면, 사람들에게 	-> 다면, 사람들에게 
12 다면, 사람들에게  	-> 면, 사람들에게 목
13 면, 사람들에게 목 	-> , 사람들에게 목재
14 , 사람들에게 목재 	->  사람들에게 목재를
15  사람들에게 목재를 	-> 사람들에게 목재를 
16 사람들에게 목재를  	-> 람들에게 목재를 가
17 람들에게 목재를 가 	-> 들에게 목재를 가져
18 들에게 목재를 가져 	-> 에게 목재를 가져오
19 에게 목재를 가져오 	-> 게 목재를 가져오게
20 게 목재를 가져오게 	->  목재를 가져오게 
21  목재를 가져오게  	-> 목재를 가져오게 하
22 목재를 가져오게 하 	-> 재를 가져오게 하고
23 재를 가져오게 하고 	-> 를 가져오게 하고 
24 를 가져오게 하고  	->  가져오게 하고 일
25  가져오게 하고 일 	-> 가져오게 하고 일을
26 가져오게 하고 일을 	-> 져오게 하고 일을 
27 져오게 하고 일을  	-> 오게 하고 일을 지
28 오게 하고 일을 지 	-> 게 하고 일을 지시
29 게 하고 일을 지시 	->  하고 일을 지시하
30  하고 일을 지시하 	-> 하고 일을 지시하고
31 하고 일을 지시하고 	-> 고 일을 지시하고 
32 고 일을 지시하고  	->  일을 지시하고 일
33  일을 지시하고 일 	-> 일을 지시하고 일감
34 일을 지시하고 일감 	-> 을 지시하고

In [7]:
batch_size = len(dataX)
batch_size

85

In [8]:
X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

# One-hot encoding, check out the shape
X_one_hot = tf.one_hot(X, num_classes)
X_one_hot  

<tf.Tensor 'one_hot:0' shape=(?, 10, 49) dtype=float32>

In [9]:
# 학습에 직접적으로 사용하지 않고 학습 횟수에 따라 단순히 증가시킬 변수를 만듭니다.
global_step = tf.Variable(0, trainable=False, name='global_step')

# Make a lstm cell with hidden_size (each unit output vector size)
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, dtype=tf.float32)

# (optional) softmax layer
X_for_softmax = tf.reshape(outputs, [-1, hidden_size])
softmax_w = tf.get_variable("softmax_w", [hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])
# All weights are 1 (equal weights)
weights = tf.ones([batch_size, seq_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)

# train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(mean_loss)
# global_step로 넘겨준 변수를, 학습용 변수들을 최적화 할 때 마다 학습 횟수를 하나씩 증가시킵니다.
train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(mean_loss, global_step=global_step)


In [10]:
# 학습모델 저장폴더 생성
MODEL_DIR = './models/rnn04'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [11]:
sess = tf.Session()
# sess.run(tf.global_variables_initializer())

saver = tf.train.Saver(tf.global_variables())

chkpoint = tf.train.get_checkpoint_state(MODEL_DIR)
if chkpoint and tf.train.checkpoint_exists(chkpoint.model_checkpoint_path):
    saver.restore(sess, chkpoint.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [12]:
batch_size, len(dataX), len(dataY)

(85, 85, 85)

In [13]:
time1 = time.time()

# for i in tqdm_notebook(range(500)):
for i in tqdm_notebook(range(10)):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)
        
time2 = time.time()  

# 최적화가 끝난 뒤, 변수를 저장합니다.
saver.save(sess, './models/rnn04/long_seq.rnn', global_step=global_step)

A Jupyter Widget

0 0 면면면면면면면면면면 3.962049
0 1 주주주면면면면면면면 3.962049
0 2 면면면면면면면면면면 3.962049
0 3 면면면면면면면면면면 3.962049
0 4 면면면면면면면면면면 3.962049
0 5 면면면면면면면면면면 3.962049
0 6 면면면면면면면줘줘줘 3.962049
0 7 면면면면면면줘줘줘면 3.962049
0 8 면면면면면줘줘줘면면 3.962049
0 9 면면면면줘줘줘면면면 3.962049
0 10 면면면면면면면면면면 3.962049
0 11 면면면줘면면면면면면 3.962049
0 12 면줘줘줘주면면면면면 3.962049
0 13 면주주주면면면면면면 3.962049
0 14 면면면면면면면면면면 3.962049
0 15 면면면면면면면면면면 3.962049
0 16 면면면면면면면면면면 3.962049
0 17 면면면면면면면면면면 3.962049
0 18 면면면면면면면면면면 3.962049
0 19 면면줘줘면면면면면줘 3.962049
0 20 면면면면면면면면줘줘 3.962049
0 21 면면면면면면면면줘줘 3.962049
0 22 면면면면면면줘줘줘줘 3.962049
0 23 면면면면면줘줘줘줘줘 3.962049
0 24 면면면주주줘줘줘줘줘 3.962049
0 25 면면주주줘줘줘줘줘줘 3.962049
0 26 주주주줘줘줘줘줘줘줘 3.962049
0 27 면면줘줘줘줘줘줘줘줘 3.962049
0 28 면줘줘줘줘줘줘줘줘줘 3.962049
0 29 면면면줘줘줘줘줘줘면 3.962049
0 30 면면면면면면면면면면 3.962049
0 31 면면주면면면면면면주 3.962049
0 32 면면면면면면면면면면 3.962049
0 33 면면면면면면면면면면 3.962049
0 34 면면면면면면면면면면 3.962049
0 35 면면면면면면줘면면면 3.962049
0 36 면면면면주주면면면줘 3.962049
0 37 면주주주주주면면줘줘 3.962049
0 38 면면면줘면면줘줘줘줘 3.962049
0 39 면면주면면면줘줘줘줘 3.962049
0 40 면면면면면

8 73            3.357915
8 74 고고         3.357915
8 75 고          3.357915
8 76 고          3.357915
8 77            3.357915
8 78 고          3.357915
8 79 고고         3.357915
8 80 고          3.357915
8 81            3.357915
8 82  경 경심심심을   3.357915
8 83 당경경경심심을    3.357915
8 84            3.357915
9 0 게          48 고          3.386359
9 49            3.386359
9 50            3.386359
9 51            3.386359
9 52 고          3.386359
9 53            3.386359
9 54            3.386359
9 55            3.386359
9 56            3.386359
9 57 고          3.386359
9 58            3.386359
9 59 고          3.386359
9 60            3.386359
9 61            3.386359
9 62            3.386359
9 63            3.386359
9 64 고          3.386359
9 65            3.386359
9 66            3.386359
9 67            3.386359
9 68            3.386359
9 69            3.386359
9 70            3.386359
9 71 고          3.386359
9 72            3.386359
9 73            3.386359
9 74 고          3.386359
9 75        

'./models/rnn04/long_seq.rnn-10'

In [14]:
chk_processting_time(time1, time2)

처리시간 : 0분 7초 경과되었습니다.


7.154566526412964

In [15]:
# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})

for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(sentence[0] + ''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')


당고                                                                                          을을 

In [17]:
print("Original sentence : \n{}".format(sentence))

Original sentence : 
당신이 배를 만들고 싶다면, 사람들에게 목재를 가져오게 하고 일을 지시하고 일감을 나눠주는 일을 하지 말라. 대신 그들에게 저 넓고 끝없는 바다에 대한 동경심을 키워줘라.


### 학습횟수를 더 늘려보자!!

In [19]:
time1 = time.time()

# for i in tqdm_notebook(range(500)):
for i in tqdm_notebook(range(100)):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        # print(i, j, ''.join([char_set[t] for t in index]), l)
        
    print(i, batch_size-1, ''.join([char_set[t] for t in index]), l)
        
time2 = time.time()  

# 최적화가 끝난 뒤, 변수를 저장합니다.
saver.save(sess, './models/rnn04/long_seq.rnn', global_step=global_step)

A Jupyter Widget

0 84 일경심을 키워줘라. 0.12067056
1 84 일경심을 키워줘라. 0.12066543
2 84 일경심을 키워줘라. 0.12064637
3 84 일경심을 키워줘라. 0.12063135
4 84 일경심을 키워줘라. 0.12062127
5 84 일경심을 키워줘라. 0.12060768
6 84 일경심을 키워줘라. 0.12059088
7 84 일경심을 키워줘라. 0.120584495
8 84 일경심을 키워줘라. 0.12057599
9 84 일경심을 키워줘라. 0.12055946
10 84 일경심을 키워줘라. 0.12054507
11 84 일경심을 키워줘라. 0.12053449
12 84 일경심을 키워줘라. 0.120528854
13 84 일경심을 키워줘라. 0.12052196
14 84 일경심을 키워줘라. 0.12051488
15 84 일경심을 키워줘라. 0.1205097
16 84 일경심을 키워줘라. 0.1205066
17 84 일경심을 키워줘라. 0.12051023
18 84 일경심을 키워줘라. 0.12051831
19 84 일경심을 키워줘라. 0.12053568
20 84 일경심을 키워줘라. 0.12055719
21 84 일경심을 키워줘라. 0.12058263
22 84 일경심을 키워줘라. 0.1205971
23 84 일경심을 키워줘라. 0.12061151
24 84 일경심을 키워줘라. 0.120593354
25 84 일경심을 키워줘라. 0.120527714
26 84 일경심을 키워줘라. 0.1204355
27 84 일경심을 키워줘라. 0.12040008
28 84 일경심을 키워줘라. 0.12043066
29 84 일경심을 키워줘라. 0.12044311
30 84 일경심을 키워줘라. 0.12040735
31 84 일경심을 키워줘라. 0.12037676
32 84 일경심을 키워줘라. 0.1203858
33 84 일경심을 키워줘라. 0.120391026
34 84 일경심을 키워줘라. 0.120366804
35 84 일경심을 키워줘라. 0.12035153
3

'./models/rnn04/long_seq.rnn-210'

In [20]:
# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})

for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(sentence[0] + ''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')


당신이 배를 만들고 싶다면, 사람들에게 목재를 가져오게 하고 일을 지시하고 일감을 나눠주는 일을 하지 말라. 대신 그들에게 저 넓고 끝없는 바다에 대한 동경심을 키워줘라.

In [21]:
print("Original sentence : \n{}".format(sentence))

Original sentence : 
당신이 배를 만들고 싶다면, 사람들에게 목재를 가져오게 하고 일을 지시하고 일감을 나눠주는 일을 하지 말라. 대신 그들에게 저 넓고 끝없는 바다에 대한 동경심을 키워줘라.


<br/><br/>
<div align="center"><font size="5" color="blue"><strong>Wow~ RNN is amazing !! </strong></font></div>
<br/>

<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>